In [ ]:
#| default_exp plane3d

# plane3D 

> Definition and operations on a three dimensional plane

In [ ]:
#|hide

%load_ext autoreload
%autoreload 2

from nbdev.showdoc import *

In [ ]:
#|export

import numpy as np
import pyransac3d 
from easydict import EasyDict as edict
from fastcore.utils import *    # to get patch

A 3D plane is defined by the implicit equation 
$$
ax + by + cz + d=0
$$
which we will express as 
$$
\tt{eq}[0]x + \tt{eq}[1]y + \tt{eq}[2]z + \tt{eq}[3]=0
$$

In [ ]:
#|export

class Plane3d:
    def __init__(self, 
                 eq: np.ndarray):  # The plane coefficients $(a,b,c,d)$=(eq[0],eq[1],eq[2],eq[3])
        "3D plane"
        self.eq = eq    
    def __str__(self): return f'<{self.eq}, (x,y,z,1)>=0'
    __repr__ = __str__

    @property
    def normal(self):
        "Return a normal vector to the plane"
        return np.array([self.eq[0], self.eq[1], self.eq[2]]) # Return the normal vector as 3D array
    
    def x_val(self, y, z):
        "Retun the x component of the plane given (y,z) coordinates. Note: will fail when $a=0$"
        return ((self.eq[2]*z + self.eq[1]*y + self.eq[3]) / (-self.eq[0]))
    
    def y_val(self, x, z):
        "Retun the x component of the plane given (x,z) coordinates. Note: will fail when $b=0$"
        return ((self.eq[0] * x + self.eq[2]*z + self.eq[3]) / (-self.eq[1]))

    def z_val(self, x, y):
        "Retun the z component of the plane given (x,y) coordinates. Note: will fail when $c=0$"
        return ((self.eq[0] * x + self.eq[1] * y + self.eq[3]) / (-self.eq[2]))
    
    @staticmethod
    def normal_to_3_points(p0: np.ndarray, # 3D array - first point
                           p1: np.ndarray, # 3D array - second point
                           p2: np.ndarray): # 3D array - third point
        "Get the normal to 3 points"
        v1 = p1 - p0         # line p1 -- p0
        v2 = p2 - p0         # line p2 -- p0 
        n = np.cross(v1, v2)
        return n

    @staticmethod
    def from_normal_and_point(n: np.array, # 3D array: Normal vectror
                              p0: np.array): # 3D array: ap 3D point
        eq = np.array([n[0], n[1], n[2], -np.dot(n,p0)])
        return Plane3d(eq)
        
    @staticmethod
    def from_3_points(p0: np.ndarray, # 3D array - first point
                      p1: np.ndarray, # 3D array - second point
                      p2: np.ndarray): # 3D array - third point
        "Construct a plane from 3 points"
        n = Plane3d.normal_to_3_points(p0,p1,p2)
        return Plane3d.from_normal_and_point(n, p0)

    @staticmethod
    def fit_plane(points, thresh=0.1,minPoints=30):
        plane1 = pyransac3d.Plane()
        best_eq, best_inliers = plane1.fit(points, thresh=thresh,minPoints=minPoints)
        return Plane3d(best_eq), best_inliers


In [ ]:
show_doc(Plane3d.x_val)
show_doc(Plane3d.y_val)

---

#### Plane3d.y_val

>      Plane3d.y_val (x, z)

Retun the x component of the plane given (x,z) coordinates. Note: will fail when $b=0$

Build the plane that passes thgrough points $(0,0,0)$, $(0,1,0)$ and $(1,1,0)$. This is the plane $z=0$

In [ ]:
p0 = np.array([0.0,0.0,0.0])
p1 = np.array([0.0,1.0,0.0])
p2 = np.array([1.0,1.0,0.0])
plane_3d = Plane3d.from_3_points(p0,p1,p2)
plane_3d

<[ 0.  0. -1. -0.], (x,y,z,1)>=0

In [ ]:
#|export

@patch
def get_span_vectors(self:Plane3d):
    "Return two vectors $v_1,v_2$ spanning the plane: independent vectors that are on the plane"
    # get two points on a plane
    if np.abs(self.eq[0]) > np.finfo(np.float32).eps:
        # a != 0
        p0 = np.array([self.x_val(0, 0),0,0])     # point on the plane with coordinates x, y=0, z=0
        p1 = np.array([self.x_val(0, 1),0,1])     # point on the plane with coordinates x, y=0, z=1
    elif np.abs(self.eq[1]) > np.finfo(np.float32).eps:
        # b != 0
        p0 = np.array([0,self.y_val(0, 0),0])     # point on the plane with coordinates x=0, y, z=0
        p1 = np.array([0,self.y_val(0, 1),1])     # point on the plane with coordinates x=0, y, z=1
    elif np.abs(self.eq[2]) > np.finfo(np.float32).eps:
        # c != 0
        p0 = np.array([0,0, self.z_val(0, 0)])     # point on the plane with coordinates x=0, y=0, z
        p1 = np.array([0,1, self.z_val(0, 1)])     # point on the plane with coordinates x=0, y=0, z

    v1 = p1 - p0                             # vector p0 --> p1 is on the plane
    v2 = np.cross(v1, self.normal)      # vector on the plane perpendicular to v1
    v1 = v1/np.linalg.norm(v1)               
    v2= v2/np.linalg.norm(v2)                

    return edict(origin=p0,v1=v1, v2=v2)

In [ ]:
plane_3d.get_span_vectors()

{'origin': array([0., 0., 0.]),
 'v1': array([0., 1., 0.]),
 'v2': array([-1.,  0.,  0.])}

## Transformations

In [ ]:
#|export

@patch
def transform_plane(self:Plane3d, transformation):
    p1 = [0, 1, self.z_val(0, 1)]
    p2 = [1, 0, self.z_val(1, 0)]
    p3 = [1, 1, self.z_val(1, 1)]

    transform_points = transformation(np.array([p1, p2, p3]))
    return Plane3d.find_eq(np.array(transform_points))

@patch
def project_3D_points(self:Plane3d,p):
    p_center = p-self.project_2d['origin']
    return np.matmul(p_center,self.project_2d['projection'].T)

@patch
def inject_2D_points(self:Plane3d,p):
    return np.matmul(p,self.project_2d['projection']) + self.project_2d['origin']
        

In [ ]:
show_doc(Plane3d.transform_plane)

---

#### Plane3d.transform_plane

>      Plane3d.transform_plane (transformation)

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()